# 📥 Ingestão de Claims para Camada Bronze

Este notebook realiza a ingestão batch de dados de claims para a camada bronze, processando **TODOS** os arquivos `claims*.csv` do volume automaticamente.

## 📋 Objetivo

- Processar todos os arquivos `claims*.csv` do diretório `/Volumes/{catalog}/00_landing/sql_server/`
- Criar/atualizar a tabela `${catalog}.${schema}.claims` na camada bronze
- Processar automaticamente múltiplos arquivos usando padrão glob

## 🔄 Fluxo de Processamento

1. **Configuração**: Define catálogo e schema via widgets ou variáveis de ambiente
2. **Leitura**: Lê todos os arquivos `claims*.csv` do volume usando padrão glob
3. **Processamento**: Infere schema automaticamente e cacheia dados
4. **Persistência**: Salva como tabela Delta na camada bronze
5. **Validação**: Mostra estatísticas da ingestão

## 📝 Parâmetros

Este notebook aceita parâmetros via **widgets** ou **variáveis de ambiente**:

- `catalog` - Nome do catálogo (ex: `smart_claims_dev`)
- `schema` - Nome do schema bronze (ex: `01_bronze`)

## ⚠️ Importante

- O notebook **reprocessa TODOS** os arquivos `claims*.csv` sempre que executado
- Ideal para reprocessamento completo quando novos arquivos chegam
- O padrão glob `claims*.csv` processa automaticamente todos os arquivos correspondentes


## 🔧 Função Helper: Obter Parâmetros

Função auxiliar para obter parâmetros via widgets (Databricks) ou variáveis de ambiente.


In [ ]:
def _get_widget_or_env(name: str, env_var: str, default: str = None) -> str:
    """
    Obtém valor de widget do Databricks ou variável de ambiente.
    
    Args:
        name: Nome do widget
        env_var: Nome da variável de ambiente
        default: Valor padrão (opcional)
    
    Returns:
        Valor do widget, variável de ambiente ou default
    """
    value = None
    
    # Tenta obter do widget do Databricks
    if "dbutils" in globals():
        try:
            value = dbutils.widgets.get(name).strip()
        except Exception:
            value = None
    
    # Se não encontrou no widget, tenta variável de ambiente
    if not value:
        import os
        value = os.getenv(env_var, "").strip()
    
    # Se ainda não encontrou, usa default ou levanta erro
    if not value:
        if default:
            return default
        raise ValueError(f"Missing configuration for {name} / {env_var}. Configure via widget ou variável de ambiente.")
    
    return value


## 📌 Passo 1: Configurar Parâmetros

Configure os parâmetros via widgets ou variáveis de ambiente. Se estiver executando manualmente, você pode definir valores padrão.


In [ ]:
# Obter parâmetros via widgets ou variáveis de ambiente
catalog = _get_widget_or_env("catalog", "CATALOG", default="smart_claims_dev")
schema = _get_widget_or_env("schema", "SCHEMA", default="01_bronze")

print(f"📂 Catálogo: {catalog}")
print(f"📂 Schema: {schema}")


## 🗂️ Passo 2: Configurar Catálogo e Schema

Seleciona o catálogo e schema onde a tabela será criada.


In [ ]:
# Configurar catálogo e schema
spark.sql(f"USE CATALOG {catalog}")
spark.sql(f"USE SCHEMA {schema}")

print(f"✅ Catálogo e schema configurados")


## 📂 Passo 3: Definir Caminhos

Define o caminho do volume e o nome da tabela de destino.


In [ ]:
# Definir caminhos
volume_path = f"/Volumes/{catalog}/00_landing/sql_server/claims*.csv"
target_table = f"{catalog}.{schema}.claims"

print(f"📁 Volume path: {volume_path}")
print(f"🎯 Tabela destino: {target_table}")
